In [1]:
import numpy as np
from helper import Point, Particle

In [2]:
n = 100
weight = 1 / n
particles = [Particle(w = weight) for i in range(n)]
capacity = 10 # max number of particles in a single cell

In [3]:
class Cell:
    def __init__(self, capacity) -> None:
        self.nleaf = 0 # number of particles in the cell
        self.leaf = np.zeros(capacity, dtype = np.int32) # array of particles in the cell
        self.clist = 0
        self.child = np.zeros(8, dtype = np.int32) # array of children
        self.parent = 0
        self.x = self.y = self.z = 0 # coord of the center of the cell
        self.r = 0 # radius of the cell
        self.multipole = np.zeros(10, dtype = np.float32)

In [4]:
root = Cell(capacity)
root.x, root.y, root.z = 0.5, 0.5, 0.5
root.r = 0.5

In [5]:
def add_child(octant, p, cells, capacity):
    cells.append(Cell(capacity)) # create a new cell and append it to the cells list
    c = len(cells) - 1 # child n.o. of the new cell
    # geometric relationship between parent and child
    cells[c].r = cells[p].r / 2
    cells[c].x = cells[p].x + cells[c].r * ((octant & 1) * 2 - 1)
    cells[c].y = cells[p].y + cells[c].r * ((octant & 2) - 1)
    cells[c].z = cells[p].z + cells[c].r * ((octant & 4) / 2 - 1)
    # establish mutual reference in the cells list
    cells[c].parent = p
    cells[p].child[octant] = c
    cells[p].clist = (cells[p].clist | (1 << octant))
    print('+++cell {} is created as a child of cell {}'.format(c, p))
    

In [6]:
def split_cell(particles, p, cells, capacity):
    print('======start the split of cell {}======'.format(p))
    # loop over the particles in the parent cell that you want to split
    for l in cells[p].leaf:
        octant = (particles[l].x > cells[p].x) + ((particles[l].y > cells[p].y) << 1) \
               + ((particles[l].z > cells[p].z) << 2)   # finding the particle's octant
        # if there is not a child cell in the particle's octant, then create one
        if not cells[p].clist & (1 << octant):
            add_child(octant, p, cells, capacity)
        # reallocate the particle into the child cell
        c = cells[p].child[octant] 
        cells[c].leaf[cells[c].nleaf] = l # append the particle into the leaf list
        cells[c].nleaf += 1 # increment the number of the leaf
        print('>>>particle {} is reallocated in cell {}'.format(l, c))
        # check if the child reach capacity
        if cells[c].nleaf >= capacity:
            split_cell(particles, c, cells, capacity)
    print('======end split cell {}======'.format(p))

In [7]:
def build_tree(particles, root, capacity):
    # set root cell
    cells = [root]       # initialize the cells list

    # build tree
    n = len(particles)
    for i in range(n):
        # traverse from the root down to a leaf cell
        curr = 0
        while cells[curr].nleaf >= capacity:
            cells[curr].nleaf += 1
            octant = (particles[i].x > cells[curr].x) + ((particles[i].y > cells[curr].y) << 1) \
                   + ((particles[i].z > cells[curr].z) << 2)
            # if there is no child cell in the particles octant, then create one
            if not cells[curr].clist & (1 << octant):
                add_child(octant, curr, cells, capacity)
            curr = cells[curr].child[octant]
        # allocate the particle in the leaf cell
        cells[curr].leaf[cells[curr].nleaf] = i # append the particle into the current cell
        cells[curr].nleaf += 1 # increment the number of particles in the current cell
        print('particle {} is stored in cell {}'.format(i, curr))
        # check whether to split or not
        if cells[curr].nleaf >= capacity:
            split_cell(particles, curr, cells, capacity)
    
    return cells

In [8]:
cells = build_tree(particles, root, capacity)

particle 0 is stored in cell 0
particle 1 is stored in cell 0
particle 2 is stored in cell 0
particle 3 is stored in cell 0
particle 4 is stored in cell 0
particle 5 is stored in cell 0
particle 6 is stored in cell 0
particle 7 is stored in cell 0
particle 8 is stored in cell 0
particle 9 is stored in cell 0
======start the split of cell 0======
+++cell 1 is created as a child of cell 0
>>>particle 0 is reallocated in cell 1
+++cell 2 is created as a child of cell 0
>>>particle 1 is reallocated in cell 2
+++cell 3 is created as a child of cell 0
>>>particle 2 is reallocated in cell 3
>>>particle 3 is reallocated in cell 1
+++cell 4 is created as a child of cell 0
>>>particle 4 is reallocated in cell 4
>>>particle 5 is reallocated in cell 2
>>>particle 6 is reallocated in cell 4
+++cell 5 is created as a child of cell 0
>>>particle 7 is reallocated in cell 5
>>>particle 8 is reallocated in cell 5
>>>particle 9 is reallocated in cell 5
======end split cell 0======
particle 10 is stored i